# SYKモデルでのSpectral Form Factor(SFF)の計算

In [2]:
##### パッケージの読み込み #####
using LinearAlgebra  #線形代数パッケージ
using SparseArrays  #疎行列化:sparse関数
using Random  #rand関数
using Plots  #プロット用パッケージ
using Statistics  #mean関数
using Distributions  #rand関数, Normal関数
using ProgressMeter  #進捗バー表示
using FLoops  #forループの並列化

In [3]:
#フェルミオンの数#
N=24;
Nc=div(N,2);
cr=[0 1
    0 0];  #生成演算子
an=[0 0
    1 0];  #消滅演算子
id=Matrix(I,2,2);  #2×2単位行列:I
id2=[-1 0
    0 1];  #id2×id2=Iとなる2×2行列

In [ ]:
function Table(M,n)  #mathematicaのTable関数を定義(行列Mをn個並べて配列にする)
      A=[]
      for i=1:n
          push!(A,M)
      end
      return A
  end

マヨラナフェルミオン$\psi_i \quad (i=1,...N)$をCrifford代数
\begin{equation}
\{\psi_i,\psi_j\}=\delta_{ij}
\end{equation}
に従う表現行列として定義する。

そのためにディラックフェルミオン$c_i \quad (i=1,...,N)$を
\begin{equation}
\{c_i,\bar{c}_j\}=\delta_{ij},\quad \{c_i,\bar{c}_j\}=0,\quad \{\bar{c}_i,\bar{c}_j\}=0
\end{equation}
という代数に従う表現行列として定義する。（$\bar{c}_i$は）\
ここで、Pauli行列$\sigma$を用いるとディラックフェルミオンは
\begin{equation}
c_i=\otimes
\end{equation}
と書ける。

In [ ]:
function K1(n)
      A=push!(Table(id,n-1),cr)
      for i=1:Nc-n
          A=push!(A,Table(id2,Nc-n)[i])
      end
      return A
  end    
  function K2(n)
      A=push!(Table(id,n-1),an)
      for i=1:Nc-n
          A=push!(A,Table(id2,Nc-n)[i])
      end
      return A
  end   
function KroneckerProduct1(n)
      A=K1(n)[1]
      for i=1:Nc-1
          A=kron(A,K1(n)[i+1])
      end
      return A
  end        
  function KroneckerProduct2(n)
      A=K2(n)[1]
      for i=1:Nc-1
          A=kron(A,K2(n)[i+1])
      end
      return A
  end        
  function c(n)
      c=sparse(KroneckerProduct1(n))
      return c
  end
  function c_d(n)
      cd=sparse(KroneckerProduct2(n))
      return cd
  end

\begin{equation}
\psi_i=\frac{1}{\sqrt{2}}\left(c_i+\bar{c}_i\right)\quad \left(\mathrm{for}\quad i=1,...,\frac{N}{2}\right),\quad
\psi_i=\frac{i}{\sqrt{2}}\left(-c_i+\bar{c}_i\right)\quad \left(\mathrm{for}\quad i=\frac{N}{2},...,N\right)
\end{equation}


In [ ]:
##### ディラックフェルミオンでマヨラナフェルミオンを定義 #####
function make_ψ()
      ψ=[]
      for n in 1:Nc
          push!(ψ,1/√2*(c(n)+c_d(n)))
      end
      for n in 1:Nc
          push!(ψ,1/√2*(-im*c(n)+im*c_d(n)))
      end
      return ψ
  end
ψ=make_ψ();

In [ ]:
function make_ψ_2()  #ψ_iとψ_jの積を先に計算しておく
    ψ_2=[ψ[i]*ψ[j] for i in 1:N, j in 1:N]
    return ψ_2
end
ψ_2=make_ψ_2();

##### 各種パラメーターの設定 #####
q=4;
J=1;
β=5;
samples=1200;

##### ハミルトニアンの生成 #####
function Hamiltonian()
    js=zeros(Float64,(N,N,N,N));
    js=rand!(Normal(0,sqrt(J^2*factorial((q-1))/(N^(q-1)))),js);
    H=Array(sum(sum(ψ_2[i,j]*sum(sum(js[i,j,k,l]*ψ_2[k,l] for l in k+1:N) for k in j+1:N-1) for j in i+1:N-2) for i in 1:N-3));
    return H
end

\begin{equation}
\mathrm{SFF} : g(t;\beta)=\left|\frac{Z(\beta+it)}{Z(\beta)}\right|^2,
\quad Z(\beta+it)=\mathrm{Tr}\left(e^{-\beta H-iHt}\right),
\quad Z(\beta)=\mathrm{Tr}\left(e^{-\beta H}\right)
\end{equation}

In [ ]:
##### |Z(β+it)|^2 #####
function SFF_u(e,β,t)
    Z_t=sum(exp.(-(β+im*t)*e))
    return Z_t*Z_t'
end

##### Z(β) #####
function get_Z(e,β)
    Z=sum(exp.(-β*e))
    return Z
end

In [ ]:
##### 時間軸 #####
ts = exp10.(range(-1,stop=5,length=10000));   #t=[10^-1,10^5], 10000等分

g_u=zeros(Float64,(samples,length(ts)));
g_d=zeros(Float64,samples);
gs=zeros(samples,length(ts));

In [ ]:
function calc_g()
    prog=Progress(samples,1);
   @floop for  i=1:samples
   A=Hamiltonian()
   e=eigvals!(A);
        g_d[i]=get_Z(e,β);
        @inbounds for (j,t) in enumerate(ts)
            g_u[i,j]=SFF_u(e,β,t);
        end
next!(prog)
end
end
println("N=$N, ","samples=$samples")
@time calc_g()
g_u_mean=mean(g_u,dims=1);  #<Z(β+it)*Z(β-it)>
g_d_mean=mean(g_d);  #<Z(β)>
gs_mean=g_u_mean/(g_d_mean)^2;  #g(t;β) = <|Z(β,t)|^2> / <Z(β)>^2
plot(ts,gs_mean[:],xaxis=:log,yaxis=:log,xlabel="t",ylabel="g(t)",title="SFF in the SYK(N=$N, $samples samples)")

In [ ]:
################## (ts, gs_mean[:]) プロット用データをtxt形式で保存 ##################
out=open("data_N=$N"*"_$samples"*"samples.txt","w") # ~.txtを作成して開く
println(out,"ts, gs_mean")
Base.print_array(out,hcat(ts,gs_mean[:])) #(ts, gs_mean)を2列にして保存
close(out) #ファイルを閉じる

In [ ]:
# サンプル保存 #
g_u_mean=g_u_mean[:]
pushfirst!(g_u_mean,g_d_mean)
out3=open("data_N=$N"*"_$samples"*"samples_gs_part1.txt","w") # ~.txtを作成して開く
println(out3,"1行目はZ(β), 2行目からはZ(β+it)*Z(β-it)")
Base.print_array(out3,g_u_mean) #(ts, gs_mean)を2列にして保存
close(out3)